# 包库引用

In [2]:
import csv, sys, os, math
import pandas as pd
import numpy as np
from numpy import argmax
import random
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve,auc  
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import pyplot
from functools import reduce
from pathlib import Path

import catboost as cb
from catboost import CatBoostClassifier
from catboost import Pool

# 数据拼接

In [3]:
workdir = os.path.abspath('.')
#cli_data = pd.read_csv('%s/BRCA_clinical.tsv'%workdir,sep='\t')
exp_data = pd.read_csv('%s/BRCA.sampleMap_HiSeqV2'%workdir,sep='\t')
gene_name = exp_data['sample']
#gene_name.to_csv('%s/gene_name.txt'%workdir,sep='\t') #写入文件保存
exp_data = exp_data.drop(labels = 'sample',axis=1).transpose()
exp_data.columns = gene_name

ylable= pd.DataFrame(pd.DataFrame(exp_data.index)[0].str.split('-').str[-1])
exp_data['ylable'] = np.where(ylable[0].str.contains("0"), 1, 0)
exp_data.to_csv('%s/BRCA.txt'%workdir,sep='\t',index='sample')

x = exp_data.iloc[:,0:20531]
x = x.drop(columns = 'ylable')
y = exp_data.iloc[:,-1].astype(int)

# ModelBuilding

In [4]:
%%time
importance_list = []
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2,random_state = 32, stratify=y)

params = {
            'loss_function': 'Logloss',
            'eval_metric': 'AUC', 
            'iterations': 10, 
            'learning_rate': 0.03,
            'random_seed': 50,
            'l2_leaf_reg': 5, 
            'bootstrap_type': 'Bernoulli', 
            #'bagging_temperature': 0,  
            'subsample': 0.6, 
            #'sampling_frequency': 'PerTree', 
            #'best_model_min_trees': 50, 
            'depth': 2,
            'grow_policy': 'SymmetricTree',
            'min_data_in_leaf': 50, 
            #'max_leaves': 12, 
            'one_hot_max_size': 4, 
            'rsm': 0.6, 
            'nan_mode': 'Max', 
            'input_borders': None,
            'boosting_type': 'Ordered', 
            #'max_ctr_complexity': 2, 
            'logging_level':'Verbose', 
            #'metric_period': 1,
            'early_stopping_rounds': 20,
            'border_count': 100, 
            'feature_border_type': 'Uniform'
        }

        #训练模型，得到特征重要性排序
model = cb.CatBoostClassifier(**params)
model.fit(x_train,y_train)
importance = model.feature_importances_
importance_list.append(importance)

0:	total: 335ms	remaining: 3.02s
1:	total: 472ms	remaining: 1.89s
2:	total: 646ms	remaining: 1.51s
3:	total: 973ms	remaining: 1.46s
4:	total: 1.14s	remaining: 1.14s
5:	total: 1.3s	remaining: 870ms
6:	total: 1.64s	remaining: 705ms
7:	total: 1.79s	remaining: 448ms
8:	total: 1.94s	remaining: 215ms
9:	total: 2.27s	remaining: 0us
CPU times: total: 32.8 s
Wall time: 5.32 s


In [5]:
df_importance = pd.DataFrame(importance_list).fillna(0)
df_importance = df_importance.rename(columns = gene_name)
df_result = df_importance.mean(axis =0).sort_values(ascending= 0)
max_importance = df_result[0]

In [6]:
df_result

ADAMTS5    29.791529
ABCA9      10.237071
PAQR4       9.250969
ITM2A       8.207556
SPC25       8.013693
             ...    
SFTA1P      0.000000
TSC22D3     0.000000
NR3C2       0.000000
NAAA        0.000000
SELS        0.000000
Length: 20530, dtype: float64

# Checking

In [7]:
#十次cv分别每次实验的特征重要性数值（无序）
df_importance

ARHGEF10L  HIF3A  RNF17  RNF10  RNF11  RNF13  GTF2IP1  REM1  MTVR2  \
0        0.0    0.0    0.0    0.0    0.0    0.0      0.0   0.0    0.0   

   RTN4RL2  ...  TULP2  NPY5R  GNGT2  GNGT1  TULP3  PTRF  BCL6B  GSTK1  SELP  \
0      0.0  ...    0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0   0.0   

   SELS  
0   0.0  

[1 rows x 20530 columns]

In [8]:
#最终的特征重要性（有序）
df_result

ADAMTS5    29.791529
ABCA9      10.237071
PAQR4       9.250969
ITM2A       8.207556
SPC25       8.013693
             ...    
SFTA1P      0.000000
TSC22D3     0.000000
NR3C2       0.000000
NAAA        0.000000
SELS        0.000000
Length: 20530, dtype: float64

In [9]:
list(df_result.index)#有序版

['ADAMTS5',
 'ABCA9',
 'PAQR4',
 'ITM2A',
 'SPC25',
 'TNS1',
 'LYVE1',
 'ACVR1C',
 'SH3TC2',
 'NNAT',
 'DZIP3',
 'HOXD3',
 'SPAG9',
 'SCN2A',
 'AIMP2',
 'CHML',
 'CCDC60',
 'PLAA',
 'OR8B12',
 'COX8A',
 'VARS2',
 'ERN2',
 'PCBD1',
 'RPS24',
 'TP53AIP1',
 'OR6N2',
 'RPS26',
 'LOC440896',
 'ERN1',
 'KBTBD11',
 'RPS27',
 'ARHGEF10L',
 'RIMBP2',
 'OPRK1',
 'UCHL3',
 'UCHL1',
 'SNORA36A',
 'CHMP1A',
 'CHMP1B',
 'UCHL5',
 'OR1E1',
 'OR1E2',
 'NDN',
 'ATRN',
 'LEAP2',
 'POU5F1',
 'PHC2',
 'SNORA36B',
 'SS18L1',
 'MTSS1L',
 'LOC145820',
 'MICAL1',
 'MICAL2',
 'C10orf2',
 'C2CD4D',
 'C2CD4C',
 'C2CD4B',
 'C2CD4A',
 'VPS28',
 'GPR176',
 'GPR174',
 'PHF2',
 'GPR173',
 'PHF7',
 'GPR171',
 'PPT1',
 'ATP6V1G1',
 'ATP6V1G2',
 'ATP6V1G3',
 'WWC3',
 'GPR179',
 'ZNF136',
 'SERP2',
 'OBP2A',
 'RNY5',
 'RUSC1',
 'C10orf4',
 'SCGB2A1',
 'ZIC2',
 'CEACAM21',
 'CEACAM20',
 'RNY4',
 'SAPS1',
 'TCHP',
 'OR6N1',
 'PDZK1',
 'TCHH',
 'TDRD3',
 'NUAK1',
 'DSC3',
 'PDGFC',
 'LOC339674',
 'SP140',
 'MGC16142',
 'ZNF

In [10]:
name=['CatBoost']
name2=range(1,len(df_result.index)+1)
test=pd.DataFrame(columns=name,index=name2,data=df_result.index)
print(test)

      CatBoost
1      ADAMTS5
2        ABCA9
3        PAQR4
4        ITM2A
5        SPC25
...        ...
20526   SFTA1P
20527  TSC22D3
20528    NR3C2
20529     NAAA
20530     SELS

[20530 rows x 1 columns]


In [11]:
test.to_csv('ctb特征排序结果.csv')